In [ ]:
import os

from bids import BIDSLayout

from glob import glob

from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function

In [ ]:
bids_dir = os.path.join('/home/brainhack','ds-supermri','bids')
output_dir = os.path.join('/home/brainhack','ds-supermri','bids','derivatives','mialsrtk')

subject = 'eyesVD46a'

layout = BIDSLayout(bids_dir)
print(layout)

In [ ]:
bg = Node(BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.index_derivatives = True
bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"]),
                          'masks': dict(suffix='mask',datatype='anat',extensions=[".nii",".nii.gz"])}

In [ ]:
from traits.api import *
from nipype.utils.filemanip import split_filename

from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

class prepareDockerPathsInputSpec(BaseInterfaceInputSpec):
    local_T2ws_paths = InputMultiPath(File(desc='input T2ws paths', mandatory = True, exists = True))
    local_masks_paths = InputMultiPath(File(desc='input masks paths', mandatory = True, exists = True))
    local_dir = Directory(mandatory=True)
    docker_dir = Directory('/fetaldata',mandatory=True)

class prepareDockerPathsOutputSpec(TraitedSpec):
    docker_T2ws_paths = OutputMultiPath(File(desc='docker T2ws paths'))
    docker_masks_paths = OutputMultiPath(File(desc='docker masks paths'))

class prepareDockerPaths(BaseInterface):
    input_spec = prepareDockerPathsInputSpec
    output_spec = prepareDockerPathsOutputSpec

    def _run_interface(self,runtime):
        return runtime
           
    def _list_outputs(self):

        outputs = self._outputs().get()
        outputs["docker_T2ws_paths"] = []
        for p in self.inputs.local_T2ws_paths:
            p = os.path.join(self.inputs.docker_dir,p.split(self.inputs.local_dir)[1].strip("/"))
            print(p)
            outputs["docker_T2ws_paths"].append(p)
        outputs["docker_masks_paths"] = []
        for p in self.inputs.local_masks_paths:
            p = os.path.join(self.inputs.docker_dir,p.split(self.inputs.local_dir)[1].strip("/"))
            print(p)
            outputs["docker_masks_paths"].append(p)
        return outputs
    

preparePaths = Node(interface=prepareDockerPaths(), name="preparePaths")  

preparePaths.inputs.local_dir = bids_dir
preparePaths.inputs.docker_dir = '/fetaldata'

wf = Workflow(name="bids_demo",base_dir=output_dir)
wf.connect(bg, "T2ws", preparePaths, "local_T2ws_paths")
wf.connect(bg, "masks", preparePaths, "local_masks_paths")

In [ ]:
import subprocess
def run(self, command, env={}, cwd=os.getcwd()):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.run(command, shell=True,
                               env=merged_env, cwd=cwd, capture_output=True)
    return process

In [ ]:
from traits.api import *
from nipype.utils.filemanip import split_filename

from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

import nibabel as nib

class BtkNLMDenoisingInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    in_file = File(desc='Input image',mandatory=True,)
    out_postfix = traits.Str("_nlm", usedefault=True)
    weight = traits.Float(0.1,desc='NLM weight (0.1 by default)')

class BtkNLMDenoisingOutputSpec(TraitedSpec):
    out_file = File(desc='Denoised image')

class BtkNLMDenoising(BaseInterface):

    input_spec = BtkNLMDenoisingInputSpec
    output_spec = BtkNLMDenoisingOutputSpec
    
    def _run_interface(self, runtime):
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        out_file = os.path.join(os.getcwd().replace(self.inputs.bids_dir,'/fetaldata'), ''.join((name, self.inputs.out_postfix, ext)))
        cmd = 'docker run --rm -u {}:{} -v "{}":/fetaldata sebastientourbier/mialsuperresolutiontoolkit btkNLMDenoising -i "{}" -o "{}" -b {}'.format(os.getuid(),os.getgid(),self.inputs.bids_dir,self.inputs.in_file,out_file,self.inputs.weight)
        try:
            print('... cmd: {}'.format(cmd))
            run(self, cmd, env={}, cwd=os.path.abspath(self.inputs.bids_dir))
        except:
            print('Failed')
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        _, name, ext = split_filename(os.path.abspath(self.inputs.in_file))
        outputs['out_file'] = os.path.join(os.getcwd(), ''.join((name, self.inputs.out_postfix, ext)))
        return outputs

class MultipleBtkNLMDenoisingInputSpec(BaseInterfaceInputSpec):
    bids_dir = Directory(desc='BIDS root directory',mandatory=True,exists=True)
    input_images = InputMultiPath(File(desc='files to be denoised', mandatory = True))
    weight = traits.Float(0.1)
    out_postfix = traits.Str("_nlm", usedefault=True)
    
class MultipleBtkNLMDenoisingOutputSpec(TraitedSpec):
    output_images = OutputMultiPath(File())

class MultipleBtkNLMDenoising(BaseInterface):
    input_spec = MultipleBtkNLMDenoisingInputSpec
    output_spec = MultipleBtkNLMDenoisingOutputSpec

    def _run_interface(self, runtime):
        for input_image in self.inputs.input_images:
            ax = BtkNLMDenoising(bids_dir = self.inputs.bids_dir, in_file = input_image, out_postfix=self.inputs.out_postfix, weight = self.inputs.weight)
            ax.run()
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['output_images'] = glob(os.path.abspath("*.nii.gz"))
        return outputs
    

nlmDenoise = Node(interface=MultipleBtkNLMDenoising(),base_dir=os.path.join(output_dir,'bids_demo'),name='nlmDenoise')
nlmDenoise.inputs.bids_dir = bids_dir
nlmDenoise.inputs.weight = 0.1
                  
wf.connect(preparePaths, "docker_T2ws_paths", nlmDenoise, "input_images")
res = wf.run()

In [ ]:
wf.write_graph()